In [1]:
import rospy
import pcl
import numpy as np
import ctypes
import struct
import os 
import open3d as o3d
from sklearn.neighbors import KDTree
from sklearn.cluster import DBSCAN 

ModuleNotFoundError: No module named 'pcl'

In [9]:
# from pyrosenv.sensor_msgs import point_cloud2

In [10]:
PCD_PATH = "pcd-20220211T040044Z-001/pcd"

pcd_file_names = os.listdir(PCD_PATH)


In [17]:
type({"X", "L"})

set

In [18]:
i = 5 
pcd1 = o3d.io.read_point_cloud(os.path.join(PCD_PATH, pcd_file_names[i]))


NameError: name 'pd' is not defined

In [12]:
pts = np.asarray(pcd1.points)

def get_current_global_pose():
    pass

def get_relative_pose():
    pass

In [13]:
# pts = []
# for point in point_cloud2.read_points(msg, skip_nans=True):
#             pt_x = point[0]
#             pt_y = point[1]
#             pt_z = point[2]
            

In [15]:
pts

array([[-3.54374468e-01, -9.40364430e-07, -9.49542001e-02],
       [-3.76423627e-01, -9.98873929e-07, -8.69041085e-02],
       [-3.69404376e-01, -9.80247705e-07, -7.18048289e-02],
       ...,
       [-3.65453094e-01,  9.69762596e-07, -5.78819960e-02],
       [-4.65884566e-01,  1.23626648e-06, -5.72033450e-02],
       [-7.52791524e-01,  1.99760007e-06, -6.58606291e-02]])

In [14]:
xd = []
yd = []
al = []
for i in range(pts.shape[0]):
    xd.append(pts[i][0])
    yd.append(pts[i][1])
    al.append(pts[i][2])
    
print("xd Max :{} Min: {}".format(max(xd), min(xd)))
print("yd Max :{} Min: {}".format(max(yd), min(yd)))
print("al Max :{} Min: {}".format(max(al), min(al)))


db = DBSCAN(eps = 0.2,metric="euclidean", min_samples = 10)
preds = db.fit_predict(pts)
n_clusters = len(set(preds)) - (1 if -1 in preds else 0)
n_noise_pts = list(preds).count(-1)

print("Number of clusters: {}".format(n_clusters))


centroids = []
c_id =0
for c_id in range(n_clusters):
    cluster_points_indexes = [ i for i in range(len(list(preds))) if preds[i] ==c_id]
    centroid =np.sum(pts[cluster_points_indexes], axis = 0)/(len(cluster_points_indexes))
    centroids.append(centroid)
    print("Centroid : {} {}".format(c_id, centroid))

print("Noisy Pts: {}".format(n_noise_pts))


xd Max :125.76383972167969 Min: -56.76069259643555
yd Max :128.32485961914062 Min: -121.89436340332031
al Max :20.228090286254883 Min: -6.788835048675537
Number of clusters: 2
Centroid : 0 [-3.89516454e-01  5.31403055e-05 -8.54207608e-02]
Centroid : 1 [-7.22214899e-01 -4.87108693e-05 -6.64537501e-02]
Noisy Pts: 4721


In [109]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pts)
o3d.visualization.draw_geometries([pcd])

In [10]:
# ros_to_pcl(pcd)

In [11]:
# import open3d as o3d
# import numpy as np
# import copy

# if __name__ == "__main__":
#     o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
#     source_raw = o3d.io.read_point_cloud("../../test_data/ICP/cloud_bin_0.pcd")
#     target_raw = o3d.io.read_point_cloud("../../test_data/ICP/cloud_bin_1.pcd")
#     source = source_raw.voxel_down_sample(voxel_size=0.02)
#     target = target_raw.voxel_down_sample(voxel_size=0.02)
#     trans = [[0.862, 0.011, -0.507, 0.0], [-0.139, 0.967, -0.215, 0.7],
#              [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]]
#     source.transform(trans)

#     flip_transform = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]
#     source.transform(flip_transform)
#     target.transform(flip_transform)

#     vis = o3d.visualization.Visualizer()
#     vis.create_window()
#     vis.add_geometry(source)
#     vis.add_geometry(target)
#     threshold = 0.05
#     icp_iteration = 100
#     save_image = False

#     for i in range(icp_iteration):
#         reg_p2l = o3d.pipelines.registration.registration_icp(
#             source, target, threshold, np.identity(4),
#             o3d.pipelines.registration.TransformationEstimationPointToPlane(),
#             o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1))
#         source.transform(reg_p2l.transformation)
#         vis.update_geometry(source)
#         vis.poll_events()
#         vis.update_renderer()
#         if save_image:
#             vis.capture_screen_image("temp_%04d.jpg" % i)
#     vis.destroy_window()

In [13]:
pcl.PointCloud()

AttributeError: module 'pcl' has no attribute 'PointCloud'

In [12]:
XYZ_cloud = pcl.PointCloud()
XYZ_cloud.from_list(pts)

AttributeError: module 'pcl' has no attribute 'PointCloud'

In [ ]:
def get_clusters(cloud, tolerance, min_size, max_size):

  tree = cloud.make_kdtree()
  extraction_object = cloud.make_EuclideanClusterExtraction()

  extraction_object.set_ClusterTolerance(tolerance)
  extraction_object.set_MinClusterSize(min_size)
  extraction_object.set_MaxClusterSize(max_size)
  extraction_object.set_SearchMethod(tree)

  # Get clusters of indices for each cluster of points, each clusterbelongs to the same object
  # 'clusters' is effectively a list of lists, with each list containing indices of the cloud
  clusters = extraction_object.Extract()
  return clusters
  

# clusters is a list of lists each list containing indices of the cloud
# cloud is an array with each cell having three numbers corresponding to x, y, z position
# Returns list of [x, y, z, color]
def get_colored_clusters(clusters, cloud):
  
  # Get a random unique colors for each object
  number_of_clusters = len(clusters)
  colors = get_color_list(number_of_clusters)

  colored_points = []

  # Assign a color for each point
  # Points with the same color belong to the same cluster
  for cluster_id, cluster in enumerate(clusters):
    for c, i in enumerate(cluster):
      x, y, z = cloud[i][0], cloud[i][1], cloud[i][2]
      color = rgb_to_float(colors[cluster_id])
      colored_points.append([x, y, z, color])
  
  return colored_points

In [ ]:
def pcl_callback(pcl_msg):

  # Convert ROS msg to PCL data
  cloud = ros_to_pcl(pcl_msg) 

  # Extract objects and table from the scene
  objects_cloud, table_cloud = split_cloud(cloud) 

  # Get a point cloud of only the position information without color information
  colorless_cloud = XYZRGB_to_XYZ(objects_cloud)
  
  # Get groups of indices for each cluster of points
  # Each group of points belongs to the same object
  # This is effectively a list of lists, with each list containing indices of the cloud
  clusters = get_clusters(colorless_cloud, tolerance = 0.05, min_size = 100, max_size = 1500)

  # Assign a unique color float for each point (x, y, z)
  # Points with the same color belong to the same cluster
  colored_points = get_colored_clusters(clusters, colorless_cloud)

  # Create a cloud with each cluster of points having the same color
  clusters_cloud = pcl.PointCloud_PointXYZRGB()
  clusters_cloud.from_list(colored_points)

  # Convert pcl data to ros messages
  objects_msg = pcl_to_ros(objects_cloud)
  table_msg = pcl_to_ros(table_cloud)
  clusters_msg = pcl_to_ros(clusters_cloud)

  # Publish ROS messages
  objects_publisher.publish(objects_msg)
  table_publisher.publish(table_msg)
  clusters_publisher.publish(clusters_msg)